In [3]:
import os
from typing import TypedDict, List, Optional, Literal
from pydantic import BaseModel, Field, AliasChoices
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END
from constants import BASE_URL, API_KEY, MODEL_NAME

# --- Схемы данных ---

class Topic(BaseModel):
    title: str = Field(description="Заголовок темы")
    tag: Optional[str] = Field(default="#general", description="Тег")
    preview: str = Field(description="Краткое описание идеи")

class TopicProposal(BaseModel):
    topics: List[Topic] = Field(description="Список уникальных тем")

class DuplicateCheck(BaseModel):
    is_duplicate: bool = Field(description="Флаг дубликата")
    reason: str = Field(description="Пояснение")

class CriticReview(BaseModel):
    is_good: bool = Field(description="Соответствует ли пост качеству и стилю")
    feedback: str = Field(description="Что нужно исправить, если пост плохой")

# --- Состояние графа ---

class AgentState(TypedDict):
    # Конфигурация источников
    config: dict 
    # Контекст
    website_context: str
    channel_archive: str
    inspiration_context: str
    objectives: str
    # Аналитика
    style_guide: str
    inspiration_notes: str
    # Контент
    suggested_topics: List[Topic]
    selected_topics: List[str]
    final_posts: List[str]
    # Метаданные для циклов
    revision_count: int

# --- Инициализация LLM ---
llm = ChatOpenAI(base_url=BASE_URL, api_key=API_KEY, model=MODEL_NAME, temperature=0.7)

# --- Агенты ---

def librarian_agent(state: AgentState):
    """Сборщик данных с учетом настроек пользователя."""
    def read_file(path, default=""):
        if os.path.exists(path):
            with open(path, "r", encoding="utf-8") as f:
                return f.read()
        return default
    print("📚 Сбор данных...")
    conf = state.get("config", {})
    print("✅ Данные собраны.")
    if not conf.get("use_website"):
        print("⚠️ Внимание: контекст сайта не используется.")
    if not conf.get("use_archive"):
        print("⚠️ Внимание: архив не используется.")
    if not conf.get("use_inspiration"):
        print("⚠️ Внимание: вдохновение не используется.")
    
    return {
        "website_context": read_file('../DATA/website_content.md') if conf.get("use_website") else "Игнорировать.",
        "channel_archive": read_file('../DATA/posts.md') if conf.get("use_archive") else "Архив пуст.",
        "inspiration_context": read_file('../DATA/inspiration.md') if conf.get("use_inspiration") else "Вдохновение не требуется.",
        "objectives": read_file('../DATA/objectives.md', "Писать интересный контент.")
    }

def stylist_agent(state: AgentState):
    """Анализирует твой личный стиль."""
    if state["channel_archive"] == "Архив пуст.":
        return {"style_guide": "Нейтральный, дружелюбный стиль."}
    print("🔍 Анализ стиля...") 
    prompt = f"Проанализируй стиль этих постов. Выдели 5 ключевых особенностей (длина предложений, использование эмодзи, обращение к аудитории, сторителлинг):\n\n{state['channel_archive']}"
    res = llm.invoke(prompt)
    print("✅ Анализ стиля завершен.")
    return {"style_guide": res.content}

def inspiration_analyst_agent(state: AgentState):
    """Вычленяет 'механики' из постов, которые тебе нравятся."""
    if "Игнорировать" in state["inspiration_context"]:
        return {"inspiration_notes": "Нет данных."}
    print("💡 Анализ вдохновения...")
    prompt = f"Это посты, которые мне нравятся. Не копируй темы, но выдели 3-5 структурных приемов (например: 'цепляющий вопрос в начале', 'список в середине', 'неожиданный вывод'):\n\n{state['inspiration_context']}"
    res = llm.invoke(prompt)
    print("✅ Анализ вдохновения завершен.")
    return {"inspiration_notes": res.content}

def planner_agent(state: AgentState):
    """Планирует темы, соединяя твой архив, цели и приемы вдохновения."""
    planner_llm = llm.with_structured_output(TopicProposal)
    
    system_msg = (
        "Ты — контент-стратег. Твоя цель: предложить новые темы для Telegram-канала.\n"
        f"ЦЕЛИ: {state['objectives']}\n"
        f"СТИЛЬ АВТОРА: {state['style_guide']}\n"
        f"ПРИЕМЫ ИЗ ВДОХНОВЕНИЯ: {state['inspiration_notes']}"
    )
    user_msg = f"Архив постов: {state['channel_archive']}\nПредложи 5 уникальных тем, которые еще не обсуждались."
    print("🗂 Планирование тем...")
    result = planner_llm.invoke([SystemMessage(content=system_msg), HumanMessage(content=user_msg)])
    print("✅ Планирование завершено.")
    return {"suggested_topics": result.topics}

def gatekeeper_agent(state: AgentState):
    """Проверяет на дубликаты."""
    check_llm = llm.with_structured_output(DuplicateCheck)
    approved = []
    print("🔐 Проверка на дубликаты...")
    for topic in state["suggested_topics"]:
        res = check_llm.invoke(f"АРХИВ: {state['channel_archive']}\nНОВАЯ ИДЕЯ: {topic.title} - {topic.preview}\nЭто дубликат?")
        if not res.is_duplicate:
            approved.append(f"{topic.title}: {topic.preview}")
    print(f"✅ Проверка завершена. Одобрено тем: {len(approved)}")  
    return {"selected_topics": approved, "revision_count": 0}

def writer_agent(state: AgentState):
    """Пишет черновик поста."""
    topic = state["selected_topics"][0] # Берем одну тему для примера
    
    # Если это повторная итерация, учитываем фидбек критика
    feedback = state.get("feedback", "")
    feedback_prompt = f"\n\nВАЖНО: Предыдущий вариант был отклонен. Исправь следующее: {feedback}" if feedback else ""

    prompt = (
        f"Напиши пост для Telegram на тему: {topic}\n"
        f"ИСПОЛЬЗУЙ СТИЛЬ: {state['style_guide']}\n"
        f"ИСПОЛЬЗУЙ ПРИЕМЫ: {state['inspiration_notes']}\n"
        f"КОНТЕКСТ: {state['website_context']}\n"
        "ПИШИ ТОЛЬКО НА РУССКОМ. Избегай канцеляризмов и 'AI-стиля' (никаких 'В современном мире', 'Важно отметить')."
        f"{feedback_prompt}"
    )
    print("✍️ Написание поста...")
    res = llm.invoke(prompt)
    print("✅ Пост написан.")
    return {"final_posts": [res.content]}

def critic_agent(state: AgentState):
    """Проверяет качество поста."""
    critic_llm = llm.with_structured_output(CriticReview)
    post = state["final_posts"][-1]
    
    prompt = (
        f"Ты — жесткий редактор. Проверь пост на соответствие стилю и целям.\n"
        f"ПОСТ:\n{post}\n\n"
        f"ЦЕЛИ: {state['objectives']}\n"
        f"СТИЛИСТИКА: {state['style_guide']}\n"
        "Если пост звучит как шаблонный текст от нейросети или скучно — дай команду переделать."
    )
    print("🧐 Критика поста...")
    res = critic_llm.invoke(prompt)
    print("✅ Критика завершена.")
    return res

# --- Логика переходов ---

def should_rewrite(state: AgentState):
    # Если критик недоволен и мы не пробовали больше 2 раз
    review = critic_agent(state)
    if not review.is_good and state.get("revision_count", 0) < 2:
        print(f"⚠️ Критик отправил на правку: {review.feedback}")
        return "rewrite"
    print(f"✅ Критик одобрил пост: {review.feedback}")
    return "end"

# --- Сборка графа ---

workflow = StateGraph(AgentState)

workflow.add_node("librarian", librarian_agent)
workflow.add_node("stylist", stylist_agent)
workflow.add_node("inspiration_analyst", inspiration_analyst_agent)
workflow.add_node("planner", planner_agent)
workflow.add_node("gatekeeper", gatekeeper_agent)
workflow.add_node("writer", writer_agent)

workflow.set_entry_point("librarian")
workflow.add_edge("librarian", "stylist")
workflow.add_edge("librarian", "inspiration_analyst")
workflow.add_edge("stylist", "planner")
workflow.add_edge("inspiration_analyst", "planner")
workflow.add_edge("planner", "gatekeeper")
workflow.add_edge("gatekeeper", "writer")

# Цикл критика
workflow.add_conditional_edges(
    "writer",
    should_rewrite,
    {
        "rewrite": "writer",
        "end": END
    }
)

app = workflow.compile()

In [4]:
# --- Запуск ---

initial_input = {
    "config": {
        "use_website": False,
        "use_archive": True,
        "use_inspiration": False # Включи/выключи здесь
    }
}

try:
    print("🚀 Запуск...")
    result = app.invoke(initial_input)
    print("\n🚀 Готовый пост:\n", result['final_posts'][-1])
except Exception as e:
    print(f"❌ Ошибка: {e}")

🚀 Запуск...
📚 Сбор данных...
✅ Данные собраны.
⚠️ Внимание: контекст сайта не используется.
⚠️ Внимание: вдохновение не используется.
🔍 Анализ стиля...
💡 Анализ вдохновения...
✅ Анализ стиля завершен.
✅ Анализ вдохновения завершен.
🗂 Планирование тем...
✅ Планирование завершено.
🔐 Проверка на дубликаты...
✅ Проверка завершена. Одобрено тем: 5
✍️ Написание поста...
✅ Пост написан.
🧐 Критика поста...
✅ Критика завершена.
✅ Критик одобрил пост: Пост соответствует стилю и целям. Он информативный, увлекательный и хорошо структурирован. Автор использует личный опыт, сравнения и конкретные примеры, что делает текст доступным и интересным для читателей. Эмодзи используются экономно и целесообразно, а обращения к аудитории создают диалоговый тон. Пост также соответствует тематике, связанной с учебой за границей и жизнью в Китае. Хорошая работа!

🚀 Готовый пост:
 **Как китайские студенты готовятся к экзаменам: от гаокао до ночных ритуалов**

Если вам кажется, что ваша сессия — это ад, то вы точн